### Preparing CFV splits for
 Retrieve available data on the disk and validate it
* Match data table with it
* Write file names to the table
* Split table into N-cross splits + separate split for test
* Use table as CSV Dataset to retrieve the image data

In [2]:
import os
import glob
import sys; sys.path.append("../")
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedGroupKFold, GroupKFold, KFold


In [3]:
nfolds = 5
test_split = True

dataset_source = "/home/oleksii/projects/ohif-orthanc-postgres-docker/datasets/zonal_segmentation/zone_dataset_2512_20240821.csv"
df = pd.read_csv(dataset_source, sep=';')

In [64]:
def get_dir_and_basename(path):

    # Split the path into directory and file name
    directory, file_name = os.path.split(path)
    # Get the last directory name
    last_directory = os.path.basename(directory)
    # Combine the last directory name and the file name
    result = os.path.join(last_directory, file_name)
    
    return result 

In [65]:
def get_alta_dataset(data_path):
    studies = os.listdir(data_path)
    records = []
    for study_id in studies:
        
        study_path = os.path.join(data_path, study_id)
        t2ws = glob.glob(os.path.join(study_path, "*tra_t2w.nrrd"))
        adcs = glob.glob(os.path.join(study_path, "*tra_adc.nrrd"))
        segs = glob.glob(os.path.join(study_path, "*.seg.nrrd"))
        
        d = {}
        d['study_orthanc_id'] = study_id
        d["t2w"] = get_dir_and_basename(t2ws[0]) if t2ws else None  
        d["adc"] = get_dir_and_basename(adcs[0]) if adcs else None  
        d["seg"] = get_dir_and_basename(segs[0]) if segs else None  
        records.append(d)
         
    ds = pd.DataFrame(records)
    print(len(ds))
    return ds

In [66]:

data_path = "/data/oleksii/Prostate-ZONE-Datasets-NRRDS/ALTA-Zone-Dataset-multimodal-preprocessed"
dataset_alta = get_alta_dataset(data_path)

# keep full cases
dataset_alta = dataset_alta[(~dataset_alta['t2w'].isnull()) & (~dataset_alta['adc'].isnull()) & (~dataset_alta['seg'].isnull())]
print(len(dataset_alta))

2501
2501


In [67]:
# merge preprocessed data with tabular one 
dataset_alta_m = pd.merge(dataset_alta, df, how="inner", on="study_orthanc_id")

In [70]:
# recreate the old split but with correct ids
dataset1260 = pd.read_csv("/home/oleksii/projects/ohif-orthanc-postgres-docker/datasets/zonal_segmentation/zone_dataset_1260_20240129.csv", sep=";")

test_split = "/home/oleksii/projects/prostate-cad/data_loading/zonal_datasets/ALTA-Zone-Dataset/test.csv"
df_test = pd.read_csv(test_split, sep=';')
test_case_ids = df_test['t2w'].str.split('/').str[-2].to_list()

splits_path = "/home/oleksii/projects/ohif-orthanc-postgres-docker/datasets/zonal_segmentation/cross_fold_splits_20230926"
os.makedirs(splits_path, exist_ok=True)

dataset1260_test = dataset1260[dataset1260['t2w_series_oid'].isin(test_case_ids)]
dataset1260_test = dataset_alta_m[dataset_alta_m['study_orthanc_id'].isin(dataset1260_test['study_orthanc_id'])]
dataset1260_test.to_csv(os.path.join(splits_path, 'test.csv'), sep=';', index=False)

for fold in range(0, 5):
    fold_path = f"/home/oleksii/projects/prostate-cad/data_loading/zonal_datasets/ALTA-Zone-Dataset/fold_{fold}"
    
    df_valid = pd.read_csv(os.path.join(fold_path, "valid.csv"), sep=';')
    df_valid['t2w_series_oid'] = df_valid['t2w'].str.split('/').str[-2].to_list()
    
    df_train = pd.read_csv(os.path.join(fold_path, "train.csv"), sep=';')
    df_train['t2w_series_oid'] = df_train['t2w'].str.split('/').str[-2].to_list()
    
    dataset1260_valid = dataset1260[dataset1260['t2w_series_oid'].isin(df_valid['t2w_series_oid'])]
    dataset1260_train = dataset1260[dataset1260['t2w_series_oid'].isin(df_train['t2w_series_oid'])]

    dataset1260_train = dataset_alta_m[dataset_alta_m['study_orthanc_id'].isin(dataset1260_train['study_orthanc_id'])]
    dataset1260_valid = dataset_alta_m[dataset_alta_m['study_orthanc_id'].isin(dataset1260_valid['study_orthanc_id'])]

    
    print(f"Saving fold {fold}")
    
    fold_path = os.path.join(splits_path, f'fold_{fold}')
    os.makedirs(fold_path, exist_ok=True)

    dataset1260_train.to_csv(os.path.join(fold_path, 'train.csv'), sep=';', index=False)
    dataset1260_valid.to_csv(os.path.join(fold_path, 'valid.csv'), sep=';', index=False)
    


Saving fold 0
Saving fold 1
Saving fold 2
Saving fold 3
Saving fold 4


In [39]:
dataset_alta_m[dataset_alta_m['patient_id'].isna()]

,study_orthanc_id,t2w,adc,seg,study_date,patient_id,StudyInstanceUID,t2w_ids


In [ ]:
# first, get the test set
# sgkfold = KFold(n_splits=nfolds + 1, shuffle=True, random_state=42)
# splits = list(sgkfold.split(X=dataset_alta_m))
# first, get the stratified test set
# sgkfold = GroupKFold(n_splits=nfolds + 1)
# splits = list(sgkfold.split(X=dataset_alta_m, groups=dataset_alta_m['patient_id'].to_list()))

# # split test from train! 
# dataset_test = dataset_alta_m.iloc[splits[0][1]]
# dataset_train = dataset_alta_m.iloc[splits[0][0]]


In [74]:
splits_path = "/home/oleksii/projects/ohif-orthanc-postgres-docker/datasets/zonal_segmentation/cross_fold_splits_20240925"
os.makedirs(splits_path, exist_ok=True)

# len(dataset_test)
dataset1260_test.to_csv(os.path.join(splits_path, 'test.csv'), sep=';', index=False)

dataset_train = dataset_alta_m[~dataset_alta_m['study_orthanc_id'].isin(dataset1260_test['study_orthanc_id'])]
# next, get the cross fold set
sgkfold_train = GroupKFold(n_splits=nfolds)
splits_folds = list(sgkfold_train.split(X=dataset_train, groups=dataset_train['patient_id'].to_list()))

In [75]:
df.to_csv(os.path.join(splits_path, os.path.basename(dataset_source)), sep=';', index=False) 

for i, split in enumerate(splits_folds):
    print(f"Saving fold {i}")
    
    fold_path = os.path.join(splits_path, f'fold_{i}')
    os.makedirs(fold_path, exist_ok=True)
    
    train_fold = dataset_train.iloc[split[0]]
    valid_fold = dataset_train.iloc[split[1]]
    train_fold.to_csv(os.path.join(fold_path, 'train.csv'), sep=';', index=False)
    valid_fold.to_csv(os.path.join(fold_path, 'valid.csv'), sep=';', index=False)

Saving fold 0
Saving fold 1
Saving fold 2
Saving fold 3
Saving fold 4
